In [ ]:
!pip install mne
!pip install kymatio

     |████████████████████████████████| 7.4 MB 7.7 MB/s 
     |████████████████████████████████| 93 kB 1.1 MB/s 


In [ ]:
from glob import glob 
"""the glob module is used to retrieve files/pathnames matching a specified pattern"""
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
all_file_path = glob('/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/*.edf') 

In [ ]:
all_file_path

['/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/h01.edf',
 '/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/h02.edf',
 '/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/s01.edf',
 '/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/s02.edf']

In [ ]:
# Sperate the healthy patients
healthy_file_path = [i for i in all_file_path if 'h' in i.split("/")[6]]
patient_file_path = [i for i  in all_file_path if 's' in i.split("/")[6]]

print(len(healthy_file_path), len(patient_file_path))

 
print(healthy_file_path)
print(patient_file_path)

2 2
['/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/h01.edf', '/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/h02.edf']
['/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/s01.edf', '/content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/s02.edf']


In [ ]:
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    
    #will auto do the avg of all chanels and result will be the referance to all other chanels. 
    #simply the refereance that we use in EEG device to capture the signals
    data.set_eeg_reference() 
    
    data.filter(l_freq=0.5, h_freq= 45)
    
    #converting segments also called as epochs
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap=1)
    array = epochs.get_data()
    # will return numpy array
    return array

In [ ]:
sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /content/drive/MyDrive/Colab Notebooks/EEG_Sych_test_WST/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 sec)

Not setting metadata
Not setting metadata
231 matching events found
No baseline corr

In [ ]:
sample_data.shape # 1st one is  no of epochs, 2st one is channels, 3rd one is  length of signals in sec

(231, 19, 1250)

In [ ]:
%%capture 
#this one will not allow to any file to print linke above file while reading the data. 

epochs_array_healthy = [read_data(i) for i in healthy_file_path]
epochs_array_patients = [read_data(i) for i in patient_file_path]

In [ ]:
epochs_array_healthy[0].shape, epochs_array_healthy[1].shape # checking the shape of 3D array

((231, 19, 1250), (227, 19, 1250))

In [ ]:
# Creating lables, for healthy lable will be 0 and for the patients label will be 1
# this on creat the list of 231 zero's in 1D and so on till 3D. in the next list will creat 227 zero's. so it means we are labeling epochs
epochs_array_healthy_labels =  [len(i)*[0] for i in epochs_array_healthy]
epochs_array_patients_labels =  [len(i)*[1] for i in epochs_array_patients]

In [ ]:
len(epochs_array_healthy_labels), len(epochs_array_patients_labels)

(2, 2)

In [ ]:
#combing the data of healthy and patients
data_list = epochs_array_healthy + epochs_array_patients
#combing the labels of healthy and patients
label_list = epochs_array_healthy_labels + epochs_array_patients_labels

In [ ]:
#Creating the group for the each subject  
# value of i will be the zero and will increase by 1  and value in the j will be the first value of data list and increase gradually
# for first iteration [i] * len(j)  value of i = 0 and if value of j=10 then it will create the list of 10 zeros like this [0,0,0,0,0,0,0,0,0,0] 
# for second iteration [i] * len(j)  value of i = 1 and if value of j=20 then it will create the list of 20 one's like this [1,1,1,1,1,1,1,1,1,1,....]
# and so on
group_list = [[i] * len(j) for i, j in enumerate(data_list)]
len(group_list)

4

In [ ]:
# Therefore we have created the group for the each of subject
print(group_list[0])
print(group_list[1])
print(group_list[2])
print(group_list[3])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
print(group_list)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
type(data_list)

list

In [ ]:
# now we have to convert labels from list to array 
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)

group_array = np.hstack(group_list)
print(data_array.shape, label_array.shape, group_array.shape)

(955, 19, 1250) (955,) (955,)


In [ ]:
#Storing the label array for the further use in .numpy format

import os
file_path = '/content/drive/MyDrive/Colab Notebooks/DemoDataset/Labels array numpy file/'
np.save(os.path.join(file_path, 'label_array.npy'), label_array)

In [ ]:
print(data_array[954])
print(label_array[954])
print(group_array[954])

[[ 8.12261537e-07  3.57247466e-07  9.58987306e-07 ... -8.47032947e-22
   4.23516474e-22 -2.11758237e-22]
 [-4.50765277e-06 -3.63177345e-06 -3.28857879e-06 ... -1.27054942e-21
   2.11758237e-21  1.16467030e-21]
 [-3.19418036e-06 -3.30971854e-06 -4.86861060e-06 ...  0.00000000e+00
   2.11758237e-22 -7.67623608e-22]
 ...
 [ 6.81826540e-07  3.58649391e-07 -1.55665047e-07 ...  2.11758237e-21
   6.35274710e-22  8.47032947e-22]
 [ 1.67595487e-06  1.82016955e-06  1.84327781e-06 ...  1.11173074e-21
   1.58818678e-22 -1.16467030e-21]
 [ 3.36543787e-06  4.37132756e-06  5.61282836e-06 ...  1.48230766e-21
   8.47032947e-22  5.29395592e-22]]
1
3


In [ ]:
# Testing the one features 
# How to extract features from the raw data
# Extract only one feature, then shape will be 7201, 19*1 if 10 then sahape will be 7201, 19*10

# extrac feature as mean
# axis = -1 because I have to extract from 1250----this is length of signals
np.mean(data_array, axis = -1).shape


(955, 19)

# Features Extraction Methods Algo Only

## Wavelet Scattering Transform

In [ ]:
def wst_2D(data):
  #import lib
  from kymatio.sklearn import Scattering2D
  """
  # Syntax
  #class kymatio.sklearn.Scattering2D(J, shape, L=8, max_order=2, pre_pad=False, backend='numpy', out_type='array')

  #Bases: kymatio.frontend.sklearn_frontend.ScatteringTransformerMixin, kymatio.numpy.ScatteringNumPy2D

  #The 2D scattering transform : The scattering transform computes two wavelet 
                                    #transform followed by modulus non-linearity. It can be summarized as

  S_J x = [S_J^{(0)} x, S_J^{(1)} x, S_J^{(2)} x]

"""
  # Set the parameters of the scattering transform.
  # J (int) – Log-2 of the scattering scale.
  J= 3

  # here 19 is channels and 1250 sec = 20 min is len of signal
  #shape (tuple of ints) – Spatial support (M, N) of the input

  # Define a Scattering2D object.
  S = Scattering2D(J, shape=(19,1250))

  #Calculate the scattering transform.
  wst = S.scattering(data)
  return wst

In [ ]:
# Stored the Extracted Features into .numpy file
WST_features = wst_2D(data_array)
WST_features.shape

(955, 217, 2, 156)

In [ ]:
import os
file_path = '/content/drive/MyDrive/Colab Notebooks/DemoDataset/Features Algo Numpy file/'
np.save(os.path.join(file_path, 'wst_features.npy'), WST_features)

## Fast Fourier Transform (FFT)


In [ ]:
def fft(data, dct_type = 2):
  from scipy.fft import dct, idct
  #The DCT-II, will get the discreet cosine transform
  #The DCT-II and DCT-III are each other’s inverses, 
  #so for an orthonormal transform we return back to the original signal.
  dct_fft = dct(dct(data, type = dct_type, norm = 'ortho'), type = 3, norm = 'ortho')
  #get inverse dct
  in_dct_fft = idct(dct(data, type = dct_type), type = 2)
  return dct_fft, in_dct_fft

In [ ]:
dct_fft_features, in_dct_fft_features = fft(data_array)
print(dct_fft_features.shape)
print(in_dct_fft_features.shape)

(955, 19, 1250)
(955, 19, 1250)


In [ ]:
import os
file_path = '/content/drive/MyDrive/Colab Notebooks/DemoDataset/Features Algo Numpy file/'
np.save(os.path.join(file_path, 'dct_fft_features.npy'), dct_fft_features)
np.save(os.path.join(file_path, 'in_dct_fft_features.npy'), in_dct_fft_features)

## yule_Walke Method

In [ ]:
# Model Implementation documents: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.yule_walker.html
# Model Notes and Artical/paper: http://stat.wharton.upenn.edu/~steele/Courses/956/Resource/YWSourceFiles/WhyNotToUseYW.pdf
def yule_Walke(data):
  # import lib
  import statsmodels.api as sm
  rho, sigma = sm.regression.yule_walker(data, order=len(data),method="mle")
  """
  Returns:
          rho:ndarray
              AR(p) coefficients computed using the Yule-Walker method.

          sigma:float
              The estimate of the residual standard deviation.
  """
  return rho,sigma


In [ ]:
# Taking too much time due to 1D array.
"""rho_features, sigma_std = yule_Walke(data_array.flatten()) # passing 1D array which has been converted from 3D to 1D
print(rh0_features.shape)
print(sigma_std)

"""

'rho_features, sigma_std = yule_Walke(data_array.flatten()) # passing 1D array which has been converted from 3D to 1D\nprint(rh0_features.shape)\nprint(sigma_std)\n\n'

## wavelet transform.

### Continuous wavelet transform.

In [ ]:
# Model implementation doc: https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.signal.cwt.html
!pip install scipy
def cwt(data, widths = None):
  
  from scipy import signal
  # If by defualt length is none then we have to calculate manually
  if widths is None:
    widths = np.arange(1, len(data) +1)
    widths = np.arange(1, 2)
  sig  = np.cos(2 * np.pi * 7 * data) + signal.gausspulse(data - 0.4, fc=2)
  cwt_matr = signal.cwt(sig, signal.ricker, widths)

  return cwt_matr


In [ ]:
"""cwt_matr_features = cwt(data_array)
print(cwt_matr_features.shape)"""



'cwt_matr_features = cwt(data_array)\nprint(cwt_matr_features.shape)'

In [ ]:
import torch

### discrete wavelet transform

In [ ]:

def dwt(data, wavelet_name='db1', mode=None):
  import pywt
  # source : https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html
  # returns a tuple representing [estimate, coefficients]
  if mode is None:
    dwt_coff = pywt.dwt(data, wavelet_name)
  else:
    dwt_coff = pywt.dwt(data, wavelet_name, mode=mode)
  return dwt_coff

In [ ]:
print(data_array.shape)
dwt_coff_features = dwt(data_array)
dwt_coff_features=np.asarray(dwt_coff_features, order="C")
dwt_coff_features.shape

(955, 19, 1250)


(2, 955, 19, 625)

In [ ]:
import os
file_path = '/content/drive/MyDrive/Colab Notebooks/DemoDataset/Features Algo Numpy file/'
np.save(os.path.join(file_path, 'dwt_coff_features.npy'), dwt_coff_features)

# Combinig all Features Extracted from Algo 

# LogisticRegression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
# split the data into the test and train
xtrain, xtest, ytrain, ytest = train_test_split(features_array,label_array,test_size=0.25, shuffle=True)

In [ ]:
# import lib
from sklearn.preprocessing import StandardScaler
# applying the scaler
sc_x = StandardScaler()
#fiting the train data for the transformation for model 
xtrain = sc_x.fit_transform(xtrain) 
#fiting the test data for the transformation for model 
xtest = sc_x.transform(xtest)

In [ ]:
# import lib
from sklearn.linear_model import LogisticRegression
# applying the classifier
classifier = LogisticRegression(random_state = 0)
# allpying the model
classifier.fit(xtrain, ytrain)

In [ ]:
# applying the data for the prediction
y_pred = classifier.predict(xtest)

In [ ]:
# cheking the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, y_pred)

print ("Confusion Matrix : \n", cm)


In [ ]:
cm[1][1]

In [ ]:
# cheking the accuracy of model
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(ytest, y_pred))

# RandomForestClassifier

In [ ]:
# 

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(xtrain,ytrain)

y_pred=clf.predict(xtest)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(ytest, y_pred))

In [ ]:
# cheking the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, y_pred)

print ("Confusion Matrix : \n", cm)
